In [1]:
!wget -O country.csv https://www.artlebedev.ru/country-list/tab/

--2021-06-13 14:52:34--  https://www.artlebedev.ru/country-list/tab/
Resolving www.artlebedev.ru (www.artlebedev.ru)... 195.218.200.21
Connecting to www.artlebedev.ru (www.artlebedev.ru)|195.218.200.21|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 30505 (30K) [text/plain]
Saving to: ‘country.csv’

country.csv         100%[===================>]  29.79K  --.-KB/s    in 0.04s   

2021-06-13 14:52:34 (694 KB/s) - ‘country.csv’ saved [30505/30505]



In [17]:
!python -m spacy download u_core_news_md


✘ No compatible model found for 'u_core_news_md' (spaCy v2.3.0).



In [2]:
import pandas as pd
import numpy as np
import os
import json
import re

pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', 100)

### Data Path

In [3]:
data_path = '/storage1/ryazantsev/AminoEmbeddings/TenderHack/SkuChangeRequests.csv'

In [4]:
columns_ru = ['Номер заявки', 'Форма заявки', 'Наименование', 'Единица измерения', 'Изображение', 'Классификация ГОСТ/ТУ', 'Описание', 'Статус', 'Причина отказа', 'Комментарий', 'Ид оферты', 'Наименование оферты', 'Артикул оферты', 'Регион поставки', 'Срок поставки в днях От', 'Срок поставки в днях До', 'Доступное количество От', 'Доступное количество До', 'Исходные характеристики', 'Категория оферты', 'Категория справочника', 'Вид продукции', 'Количество эталонных утвержденных характеристик в категории', 'Количество использованных поставщиком эталонных утвержденных характеристик']
columns_en = ['application_num', 'application_form', 'name', 'measurement_unit', 'image', 'gost_calss', 'description', 'status', 'reason', 'comment' , 'offer_id', 'offer_name', 'offer_number', 'delivery_region', 'delivery_from','delivery_to', 'quantity_from', 'quantity_to', 'init_char', 'offer_category', 'directory_category', 'product_type', 'number_characteristics', 'number_characteristics_supplier']
columns_translator = {ru: eng for ru, eng in zip(columns_ru, columns_en)}

In [5]:
df = pd.read_csv(data_path, sep=',')

#### Перевод колонок

In [6]:
df = df.rename(columns=columns_translator)

### Формирование датасета

In [7]:
REASON_NUM = 55


def sep_reason(reason: str):
    if pd.notna(reason):
        return reason.split(';')
    return reason

def get_reason_freq(df: pd.DataFrame):
    reasons_df = df.reason.dropna().apply(sep_reason)
    reas = [item for sublist in reasons_df.to_list() for item in sublist]
    reas = pd.DataFrame(reas, columns=['reason'])
    return reas.reason.value_counts()

def get_reason_dict(df: pd.DataFrame):
    reasons_df = df.reason.dropna().apply(sep_reason)
    reas = [item for sublist in reasons_df.to_list() for item in sublist]
    reas = pd.DataFrame(reas, columns=['reason'])
    reject_reasons = {rej: i for i, rej in  enumerate(reas.reason.value_counts().index.to_list())}
    return reject_reasons

def get_reject_reason_vector_index(reason, reject_reasons):
    if isinstance(reason, list):
        return [reject_reasons.get(reas, None) for reas in reason]
    return []
    
def get_reject_reason_vector(reason_index):
    vec = np.zeros(REASON_NUM, dtype=int)
    vec[reason_index] = 1
    return vec

def get_reject_comment_vector(reason_index):
    vec = np.zeros(REASON_NUM, dtype=int)
    vec[reason_index] = 1
    return vec

def reject_index(reject_vec):
    return pd.Series(data=reject_vec, index=[f'reason_{i}' for i in range(len(reject_vec))])

def get_right_df(df: pd.DataFrame):
    df['status_bit'] = df.status.apply(lambda x: 0 if x == 'Принята' else 1)
    df.loc[((df.status_bit == 0) & (df.comment.notna())), 'comment'] = np.nan
    df.loc[((df.status_bit == 0) & (df.reason.notna())), 'reason'] = np.nan
    reject_reasons = get_reason_dict(df)
    df['reason_list'] = df.reason.apply(sep_reason)
    df['reason_index'] = df['reason_list'].apply(lambda x: get_reject_reason_vector_index(x, reject_reasons))
    df['reason_vec'] = df['reason_index'].apply(get_reject_reason_vector)
    df['reason_count'] = df['reason_vec'].apply(sum)
    reason_indx_df = df.reason_vec.apply(reject_index)
    df = pd.concat([df, reason_indx_df], axis=1)
    df['offer_subcategory'] = df.offer_category.apply(lambda x: x.split('-')[1])
    df = df.drop(columns=['product_type'])
    df = df.drop(index=df[(df.status_bit == 1) & (df.comment.isna()) & (df.reason.isna())].index)
    df = df.dropna(subset=['name'])
    return df
    

In [8]:
df = get_right_df(df)

In [11]:
def parse_init_char_names(df):
    def get_names(init_char):
        return [item['name'].lower().capitalize() if item['name'] else item['name'] for item in json.loads(init_char)]

    names = df.init_char.apply(get_names)
    return names

def parse_init_char_values(df):
    def get_value(init_char):
        return [item['value'] for item in json.loads(init_char)]

    values = df.init_char.apply(get_value)
    return values

def parse_init_char_units(df):
    def get_unit(init_char):
        items = json.loads(init_char)
        if items:    
            if items[0]['name'] == "Производитель":
                items = items[1:]
        return [item.get('unitCode', None) for item in items]

    units = df.init_char.apply(get_unit)
    return units

#### Имена характеристик по частоте встречаемости

char_names = parse_init_char_names(df)
char_names = [item for sublist in char_names.to_list() for item in sublist]

char_df = pd.DataFrame(char_names, columns=['char']).dropna()

char_df = char_df.char.apply(lambda x: x).value_counts().to_frame().reset_index()

#### Значения характеристик

char_values = parse_init_char_values(df)
# char_values = [item for sublist in char_values.to_list() for item in sublist]

# char_values_df = pd.DataFrame(char_values, columns=['char']).dropna()

# char_values_df = char_values_df.char.apply(lambda x: x.lower()).value_counts().to_frame().reset_index()

#### Ед.из. характеристик по частоте встречаемости

char_units = parse_init_char_units(df)
char_units = [item for sublist in char_units.to_list() for item in sublist]

char_units_df = pd.DataFrame(char_units, columns=['char']).dropna()

char_units_df = char_units_df.char.apply(lambda x: x.lower()).value_counts().to_frame().reset_index()

In [12]:
df['char_names'] = parse_init_char_names(df)
df['char_values'] = parse_init_char_values(df)
df['char_units'] = parse_init_char_units(df)

### Парсинг описания и попытка вытащить от туда характеристики

In [9]:
def description_parser(description):
    found_chars = []
    if pd.notna(description):
        chars_names = re.findall(r'([А-Я][а-я\s]+)\s*[:|-|—]\s*', description)
        chars_val = re.split(r'[А-Я][а-я\s]+\s*[:|-|—]\s*', description)
        if len(chars_val) > len(char_names):
            chars_val = chars_val[1:]
        for char, val in zip(chars_names, chars_val):
            if char.lower() in char_df['index'].to_list():
                found_chars.append({'name': char.lower().capitalize(), 'value': val})
        if found_chars:
            return found_chars
    return np.nan

In [10]:
df['descr_char'] = df.description.apply(description_parser)

NameError: name 'char_names' is not defined

In [18]:
from typing import List, Optional, Union

In [19]:
class BaseStep:
    def __init__(
        self,
        name: str = 'BaseStep',
        columns: List = [],
        pos_status: str = '',
        neg_status: str = '',
    ):
        self.__pre_init__()
        self.name = name
        self.columns = columns
        self.pos_status = pos_status
        self.neg_status = neg_status
        self.__post_init__()
        
    def __pre_init__(self,):
        pass
    
    def __post_init__(self,):
        pass
    
    def _eval_function(self, args):
        return args
    
    def evaluate(self, df: pd.DataFrame, return_status: bool = False):
        df_eval = df[self.columns if len(self.columns) > 1 else self.columns[0]]
        if isinstance(df_eval, pd.Series):
            results = df_eval.apply(self._eval_function)
        else:
            results = df_eval.apply(self._eval_function, axis=int(len(self.columns) > 1))
        results.columns = [self.name]
        results = results.to_frame()
        if return_status:
            status = pd.DataFrame([self.pos_status if res else self.neg_status for res in results.values], index=results.index, columns=[self.name + '_status'])
            results = (results, status)
        return results

In [20]:
class Pipeline:
    def __init__(
        self,
        name: str = 'Pipeline',
        steps = [],
        pos_status: str = 'Отказать',
        neg_status: str = 'Принять',
    ):
        self.__pre_init__()
        self.name = name
        self.steps = steps
        self.pos_status = pos_status
        self.neg_status = neg_status
        self.__post_init__()
        
    def __pre_init__(self, ):
        pass
    
    def __post_init__(self, ):
        pass
    
    def evaluate(self, df: pd.DataFrame, return_status: bool = False):
        results = []
        results_status = []
        for step in self.steps:
            if return_status:
                res, status = step.evaluate(df, return_status=return_status)
                results.append(res)
                results_status.append(status)
            else:
                res = step.evaluate(df, return_status=return_status)
                results.append(res)
        results = pd.DataFrame(pd.concat(results, axis=1).any(axis=1), columns=[self.name], index=df.index)
        
        if return_status:
            results_status = pd.concat(results_status, axis=1)
            status = pd.DataFrame([self.pos_status if res else self.neg_status for res in results.values], index=results.index, columns=[self.name + '_status'])
            results = (results, status, results_status)
        return results

In [32]:
from nltk.tokenize import word_tokenize
from pyaspeller import YandexSpeller
from transliterate import translit
import regex as re
import pandas as pd
import numpy as np
from pymystem3 import Mystem
import spacy
from spacy.lang.ru.examples import sentences
import json

#nlp = spacy.load("ru_core_news_md")

# data = pd.read_csv('table_data.csv', sep=',')
#
with open('obscene_corpus.txt', 'r') as f:
    file = f.readlines()

with open('forbidden_list.txt', 'r') as f:
    fb_file = f.readlines()


#preprocessing of vocabulary of obscene words
def vocab_lower(vocabulary: list):
    list_of_lower_words = []
    for word in vocabulary:
        list_of_lower_words.append(word.lower().replace('\n',''))
    return list_of_lower_words

vocab_obscene = vocab_lower(file)
vocab_forbidden = vocab_lower(fb_file)

# Токенизация текста
def tokenize_text(text: str):
    tokens_list = word_tokenize(text)
    return tokens_list


# Приведение текста к нижнему регистру
def lower_text(text: str):
    return (text.lower())


# Транслитерация текста (латиница -> кириллица)
def translit_chars(text: str):
    return (translit(text, 'ru', reversed=False))


# Удаление знаков препинания
def re_gex(text: str):
    nabor = re.compile(r'[.+-,!@"*#$%^&)(|\/?=_:;]')
    text_clean = nabor.sub(r' ', text)
    return text_clean


# Функция препроцессинга текста
def text_preprocess(text: str):
    '''Функция препроцессинга текста: токенизация,
        приведение к нижнему регистру,
        транслитерация и удаление знаков'''

    tokens_list = tokenize_text(lower_text(translit_chars(re_gex(text))))
    return tokens_list


# Исправление опечаток
def correct_typos(tokens_list: list):
    '''Функция принимает на вход список токенов и заменяет грамматически неверные слова на верные.
    На выходе получается список той же длины но со словами без опечаток'''

    pel = YandexSpeller()

    correct_tokens_list = []
    for word in tokens_list:
        correct_tokens_list.append(pel.spelled(word))

    assert len(tokens_list) == len(correct_tokens_list)

    return correct_tokens_list


# проверка на наличие опечаток в тексте
def check_typos(text: str):
    '''В данной функции сравнивается список токенов исходного текста
    со списком токенов прогнанных через функция проверки опечаток:
    в случае совпадения (успеха) возвращается 0, в случае несоответствия возвращается 1,
    что означает, что в тексте присутствуют опечатки'''

    tokens_list = text_preprocess(text)
    correct_list = correct_typos(tokens_list)
    for i, j in zip(tokens_list, correct_list):
        if j != i:
            return True
        else:
            return False


# проверка на наличие обсценной лексики
def check_obscene(text: str, vocab_of_obscene: list = vocab_obscene):
    '''Функция ищет соответствия токенов из текста словам из словаря обсценной лексики.'''

    tokens_list = text_preprocess(text)
    for token in tokens_list:
        if token in vocab_of_obscene:
            return True
    return False


# проверка на наличие слов, обозначающих запрещенный товар к продаже
def check_forbidden_goods(text: str, vocab_of_forbidden_goods: list = vocab_forbidden):
    '''Функция ищет соответствия токенов текста словам из списка наименований запрещенной продукции'''

    tokens_list = text_preprocess(text)
    model = Mystem()
    lemmas = []
    for word in tokens_list:
        a = model.lemmatize(word)
        if a[0] in vocab_of_forbidden_goods:
            return True
    return False


# Проверка пропусков в текстовых полях для полной формы
def check_gaps(names_values_list: list):
    lst = [None, '', ' ', '\n', '\t']
    for text_name in names_values_list:
        if text_name == (i for i in lst):
            return True
        else:
            return False


# Проверка пропусков в текстовых полях для упрощенной формы
def checks_gaps_simple(text: str):
    lst = [None, '', ' ', '\n', '\t']
    if text_name == (i for i in lst):
        return True
    else:
        return False


# Проверка на наличие в форме не менее 4-х характеристик
def check_lenght(lenght_chars: int):
    '''Обязательных полей на данный момент 3,
        в заявке необходимо указать дополнительно не менее 4 характеристик товара.
        При изменении значений параметров необходимо изменить в функции их сумму.'''

    if lenght_chars >= 7:
        return False
    else:
        return True


# Проверка на наличие в поле "Описание" количества символов не менее 50
def check_lenght_simple(text: str):
    if len(text) <= 50:
        return True
    else:
        return False

#Функция чтения из датафрема поля с характеристиками товара
def data_extract(data):
    char_dict = {}
    list_of_chars = json.loads(data['Исходные характеристики'].values[0])
    for char in range(len(list_of_chars)):
        char_dict[char] = list_of_chars[char]
    return char_dict




In [30]:
# Функция-агрегатор тестов
def checks(text: str, form_type: str,
           vocab_of_obscene: list,
           vocab_of_forbidden_goods: list,
           lenght_chars: int = None,
           names_values_list: list = None):
    '''В данной функции проводятся тесты для двух видов заявок: Полная и Упрощення.
        Разница в обработке входных данных.
        Проводимые тесты:
                - на наличие опечаток;
                - на наличие обсценной лексики;
                - на наличие наименований запрещенных товаров;
                - на пропуски в текстовых полях;
                - на количество указанных характеристик товара'''

    test_typos = check_typos(text)
    test_obscene = check_obscene(text, vocab_of_obscene)
    test_forbidden = check_forbidden_goods(text, vocab_of_forbidden_goods)

    if form_type == ['Полная']:
        test_ln = check_lenght(lenght_chars)
        test_gaps = check_gaps(names_values_list)

    else:
        test_ln = check_lenght_simple(text)
        test_gaps = check_gaps(text)

    sum_list = {'number_of_chars': test_ln,
                'gaps_in_fields': test_gaps,
                'typos_in_text_fields': test_typos,
                'obscene_in_text_fields': test_obscene,
                'forbidden_goods': test_forbidden}

    return sum_list


# Аггрегирующая функция для всех тестов
def form_checking_tests(data, vocab_of_obscene: list, vocab_of_forbidden_goods: list):
    form_type = list(data['Форма заявки'])

    if form_type == ['Полная']:
        chars = data_extract(data)
        text_all_chars = []
        names_values_list = []
        for v in chars.values():
            text = str(v['name']) + ' ' + str(v['value'])
            names_values_list.append(v['name'])
            names_values_list.append(v['value'])
            text_all_chars.append(text)

        text_all_chars_txt = ','.join(text_all_chars)
        lenght_chars = len(chars)
        results = checks(text_all_chars_txt, form_type, vocab_obscene, vocab_forbidden, lenght_chars, names_values_list)

    elif form_type == ['Упрощенная']:
        text = str(data['Описание'])
        results = checks(text, form_type, vocab_obscene, vocab_forbidden)

    else:
        raise KeyError

    # проводим тесты для наименования товара
    good_name = str(data['Наименование'])
    test_name_plur = check_plural(good_name)
    test_text_isenglish = isEnglish(good_name)
    test_name_typos = check_typos(good_name)
    test_name_obscene = check_obscene(good_name, vocab_of_obscene)
    test_name_forbidden = check_forbidden_goods(good_name, vocab_of_forbidden_goods)

    tests_list = [test_name_plur, test_text_isenglish, test_name_typos, test_name_obscene, test_name_forbidden]
    tests_names = ['plural_good_name', 'english_good_name', 'typos_in_good_name',
                   'obscene_in_good_name', 'forbidden_good']

    for name, res in zip(tests_names, tests_list):
        results[name] = res

    return results


def text_results(results: dict, provider_comments: dict, moderator_comments: dict):
    '''Функция ищет соответствия итогов результатов между словарями
    для конечного вывода для поставщика и модератора'''

    result_comment_for_provider = []
    result_comment_for_moderator = []

    for k, v in results.items():
        if v == 1:
            result_comment_for_provider.append(provider_comments[k])
            result_comment_for_moderator.append(moderator_comments[k])

    if len(result_comment_for_provider) != 0:
        result_comment_for_provider.append(provider_comments['comment_neg'])
    else:
        result_comment_for_provider.append(provider_comments['comment_pos'])
    return (
        f'Результат проверки заявки для поставщика: {result_comment_for_provider}. Результат проверки заявки для модератора: {result_comment_for_moderator}')

In [25]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import timm
import torch
import cv2
import numpy as np
from torch import nn

from transformers import BertModel, BertTokenizer, BertTokenizerFast, FeatureExtractionPipeline
import re
import numpy as np
from collections import defaultdict, namedtuple
import torchtext
import random
random.seed(666)
import torch
import torch.nn.functional as F
import pandas as pd
import timm
import math
import os
from torch.utils.data import Dataset,DataLoader
from transformers import AutoTokenizer, AutoModel

import clip
from PIL import Image
from clip.simple_tokenizer import SimpleTokenizer

from tqdm.notebook import tqdm


image_path = "/storage1/ryazantsev/AminoEmbeddings/TenderHack/sku_images/sku_images/"


# In[7]:


from albumentations import (
    Compose, Normalize, Resize,
)

from albumentations.pytorch import ToTensorV2


# In[8]:


def get_transforms_val():
    return Compose([
            Resize(224, 224),
            Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225], max_pixel_value=255.0, p=1.0),
            ToTensorV2(p=1.0),
        ], p=1.)


# In[41]:


class nfnet_rubert(nn.Module):
    def __init__(self):
        super(nfnet_rubert, self).__init__()
        self.bert  = AutoModel.from_pretrained("cointegrated/rubert-tiny")
        self.enet = timm.create_model('nfnet_l0', pretrained= False)
        self.fc1 = nn.Linear(self.bert.config.hidden_size+1000, 512)
        self.output = nn.Linear(512, 1)
 
    def forward(self, x,input_ids_title, attention_mask_title):
        x = self.enet(x)
        #print(x.shape)
        text_title = self.bert(input_ids=input_ids_title, attention_mask=attention_mask_title)[0][:,0,:]
        #text_title = mean_pooling(text_title, attention_mask_title)
        x = torch.cat([x, text_title], 1)
        x = F.relu(self.fc1(x))
        x = self.output(x)
        return x
    def load_weights(self, path):
        self.load_state_dict(torch.load(path))


# In[42]:


model_matching = nfnet_rubert()
model_matching.eval();


# In[43]:


model_matching.load_weights("/storage1/ryazantsev/AminoEmbeddings/TenderHack/models/matching_img_textF1_0.804.pt")


# In[44]:


preprocessing_matching = get_transforms_val()


# In[45]:


threshold_matching = 0.571428
tokenizer_matching = AutoTokenizer.from_pretrained("cointegrated/rubert-tiny")


# In[71]:


def inference(args):
    image = str(args[0])
    image = cv2.imread(image_path + image.split('.')[0] + ".jpg")
    image = preprocessing(image=image)['image']
    image = image.reshape(1,3,224,224)
    
    title = tokenizer(list([args[1]]), padding=True, truncation=True, return_tensors='pt')
    
    #print(title)
    
    with torch.no_grad():
        output = model_matching(image, title['input_ids'], title['attention_mask'])
        prob_out = F.sigmoid(output).detach().cpu().numpy()    
    
    return prob_out[0][0] > 0.571428


Some weights of the model checkpoint at cointegrated/rubert-tiny were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [26]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import timm
import torch
import cv2
import numpy as np
from torch import nn

import numpy as np
import random
random.seed(666)
import torch
import torch.nn.functional as F
import pandas as pd
import timm
import math
import os
from torch.utils.data import Dataset,DataLoader

from tqdm.notebook import tqdm


# In[29]:


class nfnet_watermark(nn.Module):
    def __init__(self):
        super(nfnet_watermark, self).__init__()
        self.enet = timm.create_model('nfnet_l0', pretrained= False)
        self.n_features = self.enet.head.fc.in_features
        self.enet.head.fc= nn.Linear(self.n_features, 1)
        
        
    def forward(self, x):
        x = self.enet(x)
        return x
    
    def load_weights(self, path):
        self.load_state_dict(torch.load(path))


# In[90]:


def get_transforms_val():
    return Compose([
            Resize(224, 224),
            Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225], max_pixel_value=255.0, p=1.0),
            ToTensorV2(p=1.0),
        ], p=1.)


# In[91]:


from albumentations import (
    Compose, Normalize, Resize,
)

from albumentations.pytorch import ToTensorV2


# In[92]:


image_path = "/storage1/ryazantsev/AminoEmbeddings/TenderHack/sku_images/sku_images/"


# In[52]:


model = nfnet_watermark()
model.eval();


# In[53]:


model.load_weights('/storage1/ryazantsev/AminoEmbeddings/TenderHack/models/realese_watermark.pt')


# In[93]:


preprocessing = get_transforms_val()


# In[121]:


threshold = 0.40816


# In[122]:


def inference_watermark(image):
    image = cv2.imread(image_path + str(image).split('.')[0] + ".jpg")
    image = np.array(image)
    image = preprocessing(image=image)['image']
    
    image = image.reshape(1,3,224,224)
    with torch.no_grad():
        output = model(image)
        prob_out = F.sigmoid(output).detach().cpu().numpy()
    return prob_out[0][0] > 0.40816, prob_out[0][0]


In [36]:
class CountryInManufacturer(BaseStep):
    def __pre_init__(self,):
        countries = pd.read_csv('country.csv', usecols=['name'], sep='\t')
        countries = countries.name.to_list()
        countries.append('РФ')
        countries.append('Российская Федерация')
        countries.append('Голландия')
        self.countries = [country.upper() for country in countries]
        
    def _eval_function(self, init_char):
        for item in json.loads(init_char):
            if item['name'] == "Производитель":
                return (item['value'].upper() in self.countries) if item['value'] else False
        return False

class ChechRepeatCharacteristics(BaseStep):
    def _eval_function(self, chars):
        def check_repeat_prod_type(chars):
            i = 0
            for ch in chars:
                if ch:
                    if ('Вид товаров' in ch):
                        i += 1
            return i > 1
        char_is_repeat = lambda x: len(x) > len(set(x))
        return any([char_is_repeat(chars), check_repeat_prod_type(chars)])
    
class CheckForOfferNum(BaseStep):
    def _eval_function(self, args):
        name = args[0]
        offer_number = args[1]
        if offer_number and name:
            return bool(re.findall(f"^{re.escape(str(offer_number))}", str(name)))
        return False

    
class CheckUnitsInName(BaseStep):
    def __pre_init__(self,):
        top_units = char_units_df.iloc[:50, 0].to_list()
        top_units.remove('')
        self.top_units = top_units
    
    def _eval_function(self, name):
        if name:
            return any([any(re.findall(r'[\s|\d]+' + re.escape(f"{str(sub)}") + r'[\s|\d|\.|$|/]+', str(name))) for sub in self.top_units])
    
    
class CheckUnitsInCharValues(BaseStep):
    def __pre_init__(self,):
        top_units = char_units_df.iloc[:50, 0].to_list()
        top_units.remove('')
        self.top_units = top_units
    
    def _eval_function(self, char_values):
        result = []
        if char_values:
            for val in char_values:
                result.append(any([any(re.findall(r'[\s|\d]+' + re.escape(f"{str(sub)}") + r'[\s|\d|\.|$|/]+', str(val))) for sub in self.top_units]))
        return any(result)
    
    
class CheckUnitsInCharNames(BaseStep):
    def __pre_init__(self,):
        top_units = char_units_df.iloc[:50, 0].to_list()
        top_units.remove('')
        top_units.remove('штук')
        top_units.remove('листов')
        self.top_units = top_units
    
    def _eval_function(self, char_names):
        result = []
        if char_names:
            for name in char_names:
                result.append(any([any(re.findall(r'[\s|\d]+' + re.escape(f"{str(sub)}") + r'[\s|\d|\.|$|/]+', str(name))) for sub in self.top_units]))
        return any(result)
    

class CheckNamePicMatch(BaseStep):
    def _eval_function(self, args):
        return inference(args)
    
    
class CheckWatermarks(BaseStep):
    def _eval_function(self, image):
        return inference_watermark(image)
    

class CheckTypos(BaseStep):
    def _eval_function(self, name):
        return check_typos(name)

In [37]:
pp = Pipeline(steps=[CountryInManufacturer(name='CountryInManufacturer', columns=['init_char'], pos_status='в карточке заявки в поле "Производитель" - должна быть отражена компания производитель, а не страна, в которой был произведен товар', neg_status='OK'),
                     ChechRepeatCharacteristics(name='ChechRepeatCharacteristics', columns=['char_names'], pos_status='в списке наименований характеристик не должно быть дублирования характеристик и характеристик не свойственных СТЕ', neg_status='OK'),
                     CheckForOfferNum(name='CheckForOfferNum', columns=['name', 'offer_number'], pos_status='наименование заявки не должно начинаться с артикула', neg_status='OK'),
                     CheckUnitsInCharValues(name='CheckUnitsInCharValues', columns=['char_values'], pos_status='в блоке "Характеристики СТЕ" единицы измерения расположены в поле "Значение"', neg_status='OK'),
                     CheckUnitsInName(name='CheckUnitsInName', columns=['name'], pos_status='единицы измерения расположены в поле "Наименование"', neg_status='OK'),
                     CheckUnitsInCharNames(name='CheckUnitsInCharNames', columns=['char_names'], pos_status='в блоке "Характеристики СТЕ" единицы измерения расположены в поле "Название характеристики"', neg_status='OK'),
                     #CheckNamePicMatch(name='CheckNamePicMatch', columns=['name', 'image'], pos_status='описание товара (наименование, характеристики) не совпадает с представленным товаром на изображении', neg_status='OK'),
                     #CheckWatermarks(name='CheckNamePicMatch', columns=['image'], pos_status='изображение не должно содержать логотипа, оттисков печатей, водяных знаков, QR-кода, ссылок на веб-ресурсы, а также иных документов', neg_status='OK'),
                     CheckTypos(name='CheckTypos', columns=['name'], pos_status='ошибка в наименовании СТЕ', neg_status='OK'),])


In [41]:
pp.evaluate(df.iloc[5000:5001], return_status=False)

AttributeError: 'NoneType' object has no attribute 'shape'

In [830]:
a1, a2, a3 = pp.evaluate(df, return_status=True)